# Finetuning Using Google Gemma's Model

In [4]:
!pip uninstall -y bitsandbytes peft trl accelerate transformers datasets
!pip install -U transformers accelerate peft trl bitsandbytes datasets


Found existing installation: bitsandbytes 0.49.1
Uninstalling bitsandbytes-0.49.1:
  Successfully uninstalled bitsandbytes-0.49.1
Found existing installation: peft 0.18.1
Uninstalling peft-0.18.1:
  Successfully uninstalled peft-0.18.1
Found existing installation: trl 0.27.1
Uninstalling trl-0.27.1:
  Successfully uninstalled trl-0.27.1
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: transformers 5.0.0
Uninstalling transformers-5.0.0:
  Successfully uninstalled transformers-5.0.0
Found existing installation: datasets 4.5.0
Uninstalling datasets-4.5.0:
  Successfully uninstalled datasets-4.5.0
  Using cached transformers-5.0.0-py3-none-any.whl.metadata (37 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.18.1-py3-none-any.whl.metadata (14 kB)
  Using cached trl-0.27.1-py3-none-any.whl.metadata (11 kB)
  Using cached bitsandbytes-0.49.1-py3-

In [1]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [2]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

### Prerequisites
* nf4(4-bit NormalFloat(NF4)) : https://www.kaggle.com/code/lorentzyeung/what-s-4-bit-quantization-how-does-it-help-llama2


In [3]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

Loading weights:   0%|          | 0/164 [00:00<?, ?it/s]

In [5]:
text = "WHY THE SKY IS BLUE?"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

WHY THE SKY IS BLUE?

The sky is blue because of the presence of the blue pigment in the atmosphere. The blue pigment


In [6]:
import os
import wandb

# 1. Re-enable W&B
os.environ["WANDB_DISABLED"] = "false"

# 2. Login (This will prompt you for your API key from wandb.ai/authorize)
wandb.login()

# 3. Optional: Set a specific project name
os.environ["WANDB_PROJECT"] = "Tunisia-Banking-Compliance-Gemma"

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.
wandb: Currently logged in as: medalifarhat8 (medalifarhat8-mcym) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [7]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [10]:
from datasets import load_dataset

data = load_dataset("MedAliFarhat/Tunisia-Banking-Compliance-qa")


In [11]:
data['train']['question']

Column(["Quel est l'objet principal de la présente loi ?", "Quelles sont les caractéristiques juridiques et le statut d'indépendance de la Banque Centrale de Tunisie ?", 'Quel est le statut du personnel de la Banque centrale concernant le secret professionnel ?', "Où se situe le siège de la Banque centrale et peut-elle s'implanter ailleurs ?", 'Comment le capital de la Banque centrale est-il constitué et quel est son montant ?', ...])

In [31]:
def formatting_func(example):
    """
    Corrected for Tunisia-Banking-Compliance-qa
    Returns a single string instead of a list.
    """
    # Create the structured prompt for a SINGLE example
    text = f"### Instruction: Étant donné la législation bancaire tunisienne, répondez à la question.\n"
    text += f"### Question: {example['question']}\n"
    text += f"### Réponse (Article {example['article_number']}): {example['answer']}"

    return text # FIX: No brackets here

In [28]:
!pip install -q -U trl transformers accelerate peft

In [33]:
from trl import SFTConfig, SFTTrainer

sft_config = SFTConfig(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=150,
    learning_rate=2e-4,
    # CHANGE THIS:
    fp16=False,
    bf16=True,
    # -------------
    output_dir="outputs_compliance",
    hub_token=userdata.get('HF_TOKEN'),
    max_length=512,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=sft_config,
    peft_config=lora_config,
    formatting_func=formatting_func,
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Step,Training Loss
10,2.647253
20,1.576209
30,1.339819
40,1.356717
50,1.117286
60,1.078135
70,1.018376
80,1.065754
90,0.896678
100,0.822958


TrainOutput(global_step=150, training_loss=1.1178176784515381, metrics={'train_runtime': 623.4913, 'train_samples_per_second': 0.962, 'train_steps_per_second': 0.241, 'total_flos': 606648930201600.0, 'train_loss': 1.1178176784515381})

In [ ]:
from huggingface_hub import login
login(token="")


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [48]:
repo_name = "MedAliFarhat/Tunisia-Banking-Compliance-Gemma"

trainer.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)


HfHubHTTPError: (Request ID: Root=1-697e94c9-0e322a1d0aca33e40f82af38;a366a99f-d911-4c7a-ba21-750acbb5cfc9)

403 Forbidden: You don't have the rights to create a model under the namespace "MedAliFarhat".
Cannot access content at: https://huggingface.co/api/repos/create.
Make sure your token has the correct permissions.